# FULL_01: Data Loading to Feature Engineering (Production Pipeline)

**Purpose:** Process full Guayas dataset (no sampling) with 33 optimized features  
**Source:** Consolidates w01_d01 through w02_d05 notebooks  
**Output:** `data/processed/full_featured_data.pkl`

**Key Decisions Applied:**
- DEC-014: 33 features (exclude rolling std, oil, promotion interactions)
- Full Guayas region, ALL families (no 300K sampling)

**Environment:** WSL2 Ubuntu 22.04, Python 3.11, GPU available